In [ ]:
# Copyright (c) 2012-2023, NECOTIS
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification,
# are permitted provided that the following conditions are met:
#
#  - Redistributions of source code must retain the above copyright notice,
#    this list of conditions and the following disclaimer.
#  - Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
#  - Neither the name of the copyright holder nor the names of its contributors
#    may be used to endorse or promote products derived from this software
#    without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
# ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.
# IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT,
# INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT
# NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA,
# OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
# WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.

# Authors: Emmanuel Calvet, Jean Rouat, Bertrand Reulet (advisor)
# Date: July 07th, 2023
# Organization: Groupe de recherche en Neurosciences Computationnelles et Traitement Intelligent des Signaux (NECOTIS),
# Université de Sherbrooke, Canada


In [ ]:
# Import library and models
# NB: you need to import binary_model into the python path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Standard library
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# The model
from model.binaryModel import binaryModel
from model.utils import (initRandomArchitecture, 
                        initGaussianWeights, initUniformWeights, 
                        dataPath)

In [ ]:
# The experiment
sim = 'free_evolution'
distribution_weight = 'normal'
experiment = 'statisticAttractor'
experiment = experiment + '_' + distribution_weight

N = 10000
duration = 2000
nb_reservoir = 100 # Number of reservoir par value o sigma
nb_trial = 100 # Number of different state initialization of the same reservoir
K = 16 # Connectivity degree of the graph 
meanWeight = -0.1 # Average weights for the normal distribution
I = 0.2 # Fraction of up neural state (20%) at times t=0

# List of standard deviation sigma (of the normal distribution)
# Predefined sigmas close to the phase transitions
if meanWeight<0:
    sigmas = [0.01  , 0.0133, 0.0167, 0.02, 0.0233, 0.0267, 0.03  ,
            0.0333, 0.0367, 0.04,  0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.055, 0.057, 0.06, 0.061, 
            0.0622, 0.0644, 0.065,  0.066, 0.0665, 0.0667, 0.067, 0.0675, 0.068, 0.0685, 
            0.0689, 0.069, 0.0695, 0.07  , 0.0711, 0.0733, 0.0756, 0.0778, 0.08  , 0.085 ,
            0.09  , 0.1   , 0.11  , 0.13  , 0.14  , 0.15  , 0.2   , 0.3   ,
            0.4   , 0.45  , 0.5   , 0.55  , 0.65  , 0.8   , 0.95  , 1.    ,
            2.    ] 
elif meanWeight>0:
    sigmas = [0.08 , 0.1  , 0.12 , 0.15 , 0.16 , 0.2  , 0.24 , 0.28 , 0.3  ,
            0.32 , 0.35 , 0.36 , 0.37 , 0.38 , 0.392, 0.4  , 0.42 , 0.44 ,
            0.45 , 0.48 , 0.5  , 0.52 , 0.56 , 0.6  , 0.62 , 0.64 , 0.68 ,
            0.7  , 0.72 , 0.76 , 0.8  , 0.84 , 0.88 , 0.9  , 0.92 , 0.96 ,
            1.   , 1.2  , 1.5  , 2.   ]

print(f'The model will be evaluated on {len(sigmas)} values of sigma(W):')
print('sigma(W) =', sigmas)

In [ ]:
# Create the model
optionData = {'spikeCount':True, 'indexActive':False} # Option for saving data
brain = binaryModel(N, sim=sim, experiment=experiment)
seedConnectivity = 747
initRandomArchitecture(brain, K, seed=seedConnectivity) # Fixed architecture

In [ ]:
# Function to check if reservoir is ran already

def check_if_ran(sim, experiment, N, K, nbTrial, meanWeight, sigma, seed):
    #Check network has not been run already
    pathBrain = dataPath(sim, experiment, N, K, meanWeight)
    file_path = pathBrain + f"/metadata_seed{seed}_N{N}_K{K}_T{nbTrial}_W{meanWeight}_std{sigma}.npy"
    if os.path.exists(file_path):
        print('Network already run, next...\n \n')
        return True
    else:
        return False

In [ ]:
# Run the experiment

for sigma in tqdm(sigmas):
    sigma = np.round(sigma, 5)
    optionData = {'spikeCount':True, 'indexActive':False}
    for res_idx in range(nb_reservoir):
        
        seed_res = res_idx * 100

        # Check if the network is already run
        check = check_if_ran(sim, experiment, N, K, nb_trial,meanWeight, sigma, seed_res)
        if check:
            continue

        # Create weights
        if distribution_weight == 'normal':
            initGaussianWeights(brain, meanWeight, sigma, seed=seed_res)
        elif distribution_weight == 'uniform':
            initUniformWeights(brain, meanWeight, sigma, seed=seed_res)
        
        tag = f'seed{seed_res}'
        print('-------------------')
        print('| Selected network')
        print('|', tag+f' sigma:{sigma}')
        print('-------------------')

        for t in range(nb_trial):

            # Run simulation
            print(f'Reservoir #{res_idx}, trial #{t}: sigma={sigma}')
            brain.displayInfoNetwork()
            brain.initRandomState(I, seed=seed_res) # Initialize the state of the reservoir
            brain.run(duration, **optionData)

            # Reset the network state
            brain.reset()
    
        # Save data and metadata
        brain.saveData(tag=tag, **optionData)
        brain.saveMetadata(tag=tag, )
        brain.reset(deep=True)


In [ ]:
# Classify attractor of reservoir's activities

from model.utils import classify_activity_attractors
import glob 

def classify_attractor_activity(N, K, nbTrial, meanWeight, duration=2000, plotActivity=False,
                                sim='free_evolution', experiment='attractorStatistic'):
    
    # Saving folder
    results_dir = dataPath(sim, experiment, N, K, meanWeight)

    # Load filenames
    filename = f'/metadata_seed*_N{N}_K{K}_D{duration}_T{nbTrial}_W{meanWeight}_*.npy'
    file_list = glob.glob(results_dir +  filename)
        
    for filename in file_list:

        # Load informations
        metadata = np.load(filename, allow_pickle='TRUE').item()
        directory = metadata['dataPath']
        fileData = metadata['spikeCountFile']
        stdWeight = metadata.get('stdWeights', None)
        seeds = metadata['seed']
        seed = seeds['seedWeights']
        
        print(filename)
        print('Network seed:', seed)

        # Create folder to save plot
        if plotActivity:
            result_dir = dataPath(sim, experiment, N, K, meanWeights=meanWeight, stdWeights=stdWeight)
            folderTest = 'activitySorted/'
            folder_plot = os.path.join(result_dir, folderTest)
            if not os.path.isdir(folder_plot):
                os.makedirs(folder_plot)
        
        # Commonder folder
        resDir = dataPath(sim, experiment, N, K, meanWeight)
        result_directory = os.path.join(resDir, 'activitySorted/')
        if not os.path.isdir(result_directory):
            os.makedirs(result_directory)
            
        # Load activity data
        print(directory + '/' + fileData + '.npy')
        try:
            A_trials_orig = np.load(directory + '/' + fileData + '.npy')
        except:
            print('--- NOT LOADED:', fileData)
            continue

        intermediateTimeStep = int(duration-1000)
        # Classify all trials
        idxTrial_Periodic = []
        idxTrial_Const = []
        idxTrial_NonStat = []
        idxTrial_Dead = []
        idxTrial_Chaos = []
        activityTrialChaos = []
        for j in range(nbTrial):

            # Classify the activity into attractor category
            A_trial = np.array(A_trials_orig[j])
            A_trial = A_trial[intermediateTimeStep+1:]
            label = classify_activity_attractors(A_trial, N)
            
            print(label)

            if label == "Cyclic attractor":
                idxTrial_Periodic.append(j)
            elif label == "Fixed point attractor":
                idxTrial_Const.append(j)
            elif label == "No activity":
                idxTrial_Dead.append(j)
            elif label == "Irregular attractor":
                idxTrial_Chaos.append(j)
                activityTrialChaos.append(A_trial)
            elif label == "Non trivial activity":
                idxTrial_NonStat.append(j)

            if plotActivity:
                # Figure for activity
                fig1 = plt.figure()
                ax1 = fig1.add_subplot(1, 1, 1)
                print(f'Trial #{j}')
                ax1.set_ylabel('A')
                ax1.set_xlabel('TimeStep')
                ax1.plot(A_trial, label=label)
                ax1.legend()
                sig_star = np.round(stdWeight/meanWeight, 3)
                ax1.set_title('$\sigma^{\star}$' + f'={sig_star}, seed={100*j}')
                plt.tight_layout()
                fig1.savefig(folder_plot + f'{label}-activity_trial#{j}_N{N}_K{K}_W{meanWeight}_std{stdWeight}.jpg')
                plt.close()
                
        # Store trials sorted for each sigma(W)
        statAttractors = {'cyclic':len(idxTrial_Periodic)/nbTrial, 
                          'dead':len(idxTrial_Dead)/nbTrial,
                          'nontrivial':len(idxTrial_NonStat)/nbTrial,
                          'irregular':len(idxTrial_Chaos)/nbTrial,
                          'fix':len(idxTrial_Const)/nbTrial,
                          }
        print(statAttractors)
        print('---------------')
        all_stats_attractors = {'idxPeriodic': idxTrial_Periodic,
                                'idxDead': idxTrial_Dead,
                                'nonStat': idxTrial_NonStat,
                                'idxConst': idxTrial_Const,
                                'idxIrregular': idxTrial_Chaos,
                                'statAttractors':statAttractors,
                                }
        metadata.update(all_stats_attractors)
        np.save(filename, metadata)
    
classify_attractor_activity(N, K, nb_trial, meanWeight, 
                            duration=2000, plotActivity=False,
                            sim=sim, experiment=experiment)



In [ ]:
# Convert all data into a pandas dataframe for simplfying analysis
import pandas as pd
from model.utils import save_df

def format_attractor_name(attractor_name):
    dic = {'irregular':'Irregular',
           'dead':'No-Activity',
           'fix':'Fix',
           'cyclic':'Cyclic',
           'nontrivial':'Non-Trivial'}
    return dic.get(attractor_name, None)

def make_csv_attractor(N, K, meanWeight, nbTrial,
                        sim='free_evolution',
                        experiment='attractorStatistic'):
    
    # Saving folder
    results_dir = dataPath(sim, experiment, N, K, meanWeight)
    resultFolder = os.path.join(results_dir, 'allResults/')
    if not os.path.isdir(resultFolder):
        os.makedirs(resultFolder)
    df_path = resultFolder + f'{experiment}_allReservoirs_N{N}_K{K}_W{meanWeight}_T{nbTrial}'

    # Load filenames
    filename = f'/metadata_seed*_N{N}_K{K}_D{duration}_T{nbTrial}_W{meanWeight}_*.npy'
    file_list = glob.glob(results_dir +  filename)
    
    sigmas = []
    seeds = []
    dominants = []
    for i, filename in enumerate(file_list):

        # Load informations
        metadata = np.load(filename, allow_pickle='TRUE').item()
        sigma = metadata.get('stdWeights', None)
        all_seeds = metadata['seed']
        seed = all_seeds['seedWeights']

        # Update dictionary of attractor statistics  
        sigmas.append(sigma)
        seeds.append(seed)
        stat_circuit = metadata['statAttractors']
        if i == 0:
            stat_circuits = {format_attractor_name(attractor):[] for attractor in stat_circuit.keys()}
        
        for attractor, count in stat_circuit.items():
            stat_circuits[format_attractor_name(attractor)].append(count)

        # Find dominant attractor
        stats =[prob for prob in stat_circuit.values()]
        labels = [label for label in stat_circuit.keys()]
        idx_max = np.argmax(stats)
        dominants.append(format_attractor_name(labels[idx_max]))

        # Print info
        print(f'sigma={sigma}, seed={seed}')
        print(stat_circuit)
        print('-----------------')
    
    # Save csv
    print(len(dominants), len(sigmas), len(seeds))
    dic_all_statistics = {'stdWeights':sigmas,
                          'seeds':seeds,
                          'dominant':dominants,
                            }
    print(stat_circuits)
    for attractor, count in stat_circuits.items():
        print(attractor, len(count))
        dic_all_statistics.update({attractor:count})

    df = pd.DataFrame(dic_all_statistics)
    df.to_csv(df_path)

    return df 
 
df = make_csv_attractor(N, K, meanWeight, nb_trial, sim=sim, experiment=experiment)

In [ ]:
df.head()